In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame
import random as rd

In [532]:
data_table = pd.read_csv('teste.csv', names =['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'flower'])

In [533]:
matrix_x = np.array(data_table[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])
vector_y = np.array(data_table['flower'])

In [545]:
def gaussian_discriminant_analysis(matrix_x, vector_y):
    
    classes, num_occurrences = np.unique(vector_y, return_counts=True)
    
    odds =  num_occurrences / len(vector_y)
    
    means = np.ones((len(classes), matrix_x.shape[1]) , dtype=float)
    
    covariance = np.ones((len(classes), matrix_x.shape[1], matrix_x.shape[1]), dtype=float)
        
    for k in range(len(classes)):
        
        class_element = matrix_x[vector_y==k]
        
        _sum = np.zeros(matrix_x.shape[1])
        
        for i in range (class_element.shape[0]):
            _sum += class_element[i]          
        
        means[k] = (_sum / num_occurrences[k])
        
        sum_cov = np.ones((matrix_x.shape[1], matrix_x.shape[1]), dtype=float)
        
        for i in range (class_element.shape[0]):    
            sum_cov += np.outer((class_element[i] - means[k]),(class_element[i] - means[k]))
        covariance[k] = sum_cov /(num_occurrences[k] -1)         

    return means, covariance, odds

In [546]:
means, covariance, odds = gaussian_discriminant_analysis(matrix_x, vector_y)

In [536]:
def predict(x, means, covariance, odds):
    
    num_classes = len(odds)
    
    odds_class = np.zeros(num_classes)
    
    for k in range(num_classes):
        
        determinante = np.linalg.det(covariance[k])
                
        part1 = np.log2(determinante)
        
        part2 = (np.transpose(x - means[k]).dot(np.linalg.inv(covariance[k]))).dot(x - means[k])
        
        prediction = np.log2(odds[k]) -1/2 * part1 -1/2 * part2 
        
        odds_class[k] = prediction
        
    return odds_class

In [549]:
size = (matrix_x.shape[0], len(odds))
predictions = np.ones(size, dtype=float)

for i in range(matrix_x.shape[0]):
    predictions[i] = predict(matrix_x[i], means, covariance, odds)
    for i in range(predictions.shape[0]):
        index = np.argmax(predictions[i], axis=0)
        for j in range(len(predictions[i])):
            if(j == index):
                predictions[i][index] = 1.0
            else:
                predictions[i][j] = 0.0